In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/workspaces/SebastianBot")

In [2]:
from cloud.dependencies.clients import resolve_gmail_client


gmail = resolve_gmail_client()

In [3]:
# Example: Apply a label/tag to an email

# 1. First, let's see what labels exist
labels_result = gmail._service.users().labels().list(userId='me').execute()
labels = labels_result.get('labels', [])
print("Existing labels:")
for label in labels[:5]:  # Show first 5
    print(f"  - {label['name']} (ID: {label['id']})")

print(f"\nTotal labels: {len(labels)}")


Existing labels:
  - CHAT (ID: CHAT)
  - SENT (ID: SENT)
  - INBOX (ID: INBOX)
  - IMPORTANT (ID: IMPORTANT)
  - TRASH (ID: TRASH)

Total labels: 47


In [4]:
for label in labels:
    if label['name'] == 'ToRead':
        break

In [5]:
toread_id = label['id']

In [6]:
from datetime import datetime
from sebastian.shared.gmail.query_builder import GmailQueryBuilder


mail = gmail.fetch_mails(GmailQueryBuilder().from_email('azure-noreply@microsoft.com').after_date(datetime(2026,1,22)).build())[0]

In [7]:
mail_id = mail.id

In [8]:
modify_request = {
    'addLabelIds': [toread_id],
    'removeLabelIds': []
}

result = gmail._service.users().messages().modify(
    userId='me',
    id=mail_id,
    body=modify_request
).execute()